In [1]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from IPython.display import display, IFrame, HTML

# Controller Equations

\begin{gather}
\dot{x} = Ax + Bu \\
A = \frac{\partial f}{\partial x} = \frac{\partial f}{\partial s} \quad\quad B = \frac{\partial f}{\partial u} = \frac{\partial f}{\partial i} \\
u = -Kx \\
x = s - s_{eq} \quad\quad u = i - i_{eq} \\
s = [o_x, o_y, o_z, \psi, \theta, \phi, v_x, v_y, v_z, w_x, w_y, w_z] \\
u = [\tau_x, \tau_y, \tau_z, f_z]
\end{gather}

In [2]:
o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, a_z = sym.S("o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, a_z")
tau_x, tau_y, tau_z, f_z = sym.S("tau_x, tau_y, tau_z, f_z")

\begin{gather}
s_{eq} = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] \\
i_{eq} = [0, 0, 0, g]
\end{gather}

In [3]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]

In [4]:
x = []
u = []

# 2. Derive models

## 2.1 Define symbolic variables

Define states.

In [5]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

Define inputs.

In [6]:
# gyroscope measurements - components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# z-axis accelerometer measurement - specific force (meters / second^2)
a_z = sym.symbols('a_z')

Define outputs.

In [7]:
d0, d1, d2, d3, d4, d5, d6, d7, n_x, n_y, r = sym.symbols('d0, d1, d2, d3, d4, d5, d6, d7,n_x, n_y, r')
a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 = sym.symbols('a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7 ')

Define parameters.

In [8]:
g, k_flow = sym.symbols('g, k_flow')

Create linear and angular velocity vectors (in coordinates of the body frame).

In [9]:
v_01in1 = sym.Matrix([[v_x], [v_y], [v_z]])
w_01in1 = sym.Matrix([[w_x], [w_y], [w_z]])

## 2.2 Define kinematics of orientation

### 2.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [10]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [11]:
R_1in0 = Rz * Ry * Rx

### 2.2.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [12]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [13]:
N = sym.simplify(Ninv.inv())

## 2.3 Derive equations of motion

Ratio of net thrust to mass in terms of z-axis accelerometer measurement.

In [14]:
f_z_over_m = a_z + (w_01in1.cross(v_01in1))[2]

Ratio of forces to mass.

In [15]:
f_in1_over_m = R_1in0.T * sym.Matrix([[0], [0], [-g]]) + sym.Matrix([[0], [0], [f_z_over_m]])

Equations of motion.

In [16]:
f = sym.Matrix.vstack(
    R_1in0 * v_01in1,
    N * w_01in1,
    (f_in1_over_m - w_01in1.cross(v_01in1)),
)

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

where

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

In [17]:
f

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                          g*sin(theta) + v_y*w_z - v_z*w_y

# Observer Equations

In [18]:
o_x, o_y, o_z = sym.S("o_x, o_y, o_z") # drone position in abosulte frame -> loco pos frame
x_0, y_0, z_0 = sym.S("x_0, y_0, z_0")
x_1, y_1, z_1 = sym.S("x_1, y_1, z_1")
x_2, y_2, z_2 = sym.S("x_2, y_2, z_2")
x_3, y_3, z_3 = sym.S("x_3, y_3, z_3")
x_4, y_4, z_4 = sym.S("x_4, y_4, z_4")
x_5, y_5, z_5 = sym.S("x_5, y_5, z_5")
x_6, y_6, z_6 = sym.S("x_6, y_6, z_6")
x_7, y_7, z_7 = sym.S("x_7, y_7, z_7")

In [19]:
psi, theta, phi = sym.S("psi, theta, phi")
v_x, v_y, v_z = sym.S("v_x, v_y, v_z")

In [20]:
w_x, w_y, w_z = sym.S("w_x, w_y, w_z")
a_z = sym.S("a_z")

In [21]:
n_x, n_y, r = sym.S("n_x, n_y, r")

In [22]:
g, k_flow = sym.S("g, k_flow")

In [23]:
anchor_pos_0 = sym.sqrt((o_x-x_0)**2 + (o_y-y_0)**2 + (o_z-z_0)**2)
anchor_pos_1 = sym.sqrt((o_x-x_1)**2 + (o_y-y_1)**2 + (o_z-z_1)**2)
anchor_pos_2 = sym.sqrt((o_x-x_2)**2 + (o_y-y_2)**2 + (o_z-z_2)**2)
anchor_pos_3 = sym.sqrt((o_x-x_3)**2 + (o_y-y_3)**2 + (o_z-z_3)**2)
anchor_pos_4 = sym.sqrt((o_x-x_4)**2 + (o_y-y_4)**2 + (o_z-z_4)**2)
anchor_pos_5 = sym.sqrt((o_x-x_5)**2 + (o_y-y_5)**2 + (o_z-z_5)**2)
anchor_pos_6 = sym.sqrt((o_x-x_6)**2 + (o_y-y_6)**2 + (o_z-z_6)**2)
anchor_pos_7 = sym.sqrt((o_x-x_7)**2 + (o_y-y_7)**2 + (o_z-z_7)**2)

In [24]:
h = sym.Matrix([
    anchor_pos_0,
    anchor_pos_1,
    anchor_pos_2,
    anchor_pos_3,
    anchor_pos_4,
    anchor_pos_5,
    anchor_pos_6,
    anchor_pos_7,
    k_flow*(v_x-o_z*w_y)/o_z, # n_x
    k_flow*(v_y+o_z*w_x)/o_z, # n_y
    o_z / (sym.cos(phi)*sym.cos(theta)) # r
])
h

Matrix([
[sqrt((o_x - x_0)**2 + (o_y - y_0)**2 + (o_z - z_0)**2)],
[sqrt((o_x - x_1)**2 + (o_y - y_1)**2 + (o_z - z_1)**2)],
[sqrt((o_x - x_2)**2 + (o_y - y_2)**2 + (o_z - z_2)**2)],
[sqrt((o_x - x_3)**2 + (o_y - y_3)**2 + (o_z - z_3)**2)],
[sqrt((o_x - x_4)**2 + (o_y - y_4)**2 + (o_z - z_4)**2)],
[sqrt((o_x - x_5)**2 + (o_y - y_5)**2 + (o_z - z_5)**2)],
[sqrt((o_x - x_6)**2 + (o_y - y_6)**2 + (o_z - z_6)**2)],
[sqrt((o_x - x_7)**2 + (o_y - y_7)**2 + (o_z - z_7)**2)],
[                           k_flow*(-o_z*w_y + v_x)/o_z],
[                            k_flow*(o_z*w_x + v_y)/o_z],
[                             o_z/(cos(phi)*cos(theta))]])

In [25]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]
i = [w_x, w_y, w_z, a_z]
o = [d0,
    d1,
    d2,
    d3,
    d4,
    d5,
    d6,
    d7,
    n_x,
    n_y,
    r,
    ]
p = [g, k_flow]


In [26]:
o_x_eq = sym.symbols('o_x_eq')
o_y_eq = sym.symbols('o_y_eq')
o_z_eq = sym.symbols('o_z_eq')
s_eq = [o_x_eq, o_y_eq, o_z_eq, 0, 0, 0, 0, 0, 0]
i_eq = [0, 0, 0, g]

s_eq = [sym.nsimplify(a) for a in s_eq]
i_eq = [sym.nsimplify(a) for a in i_eq]


In [27]:
x = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z]) - sym.Matrix(s_eq)
u = sym.Matrix([w_x, w_y, w_z, a_z]) - sym.Matrix(i_eq)
y = sym.Matrix(o) - h.subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
y

Matrix([
[d0 - sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[d1 - sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[d2 - sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2)],
[d3 - sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2)],
[d4 - sqrt((o_x_eq - x_4)**2 + (o_y_eq - y_4)**2 + (o_z_eq - z_4)**2)],
[d5 - sqrt((o_x_eq - x_5)**2 + (o_y_eq - y_5)**2 + (o_z_eq - z_5)**2)],
[d6 - sqrt((o_x_eq - x_6)**2 + (o_y_eq - y_6)**2 + (o_z_eq - z_6)**2)],
[d7 - sqrt((o_x_eq - x_7)**2 + (o_y_eq - y_7)**2 + (o_z_eq - z_7)**2)],
[                                                                 n_x],
[                                                                 n_y],
[                                                         -o_z_eq + r]])

In [28]:
A = f.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
B = f.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
A, B

(Matrix([
 [0, 0, 0, 0, 0,  0, 1, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 1, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 1],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0],
 [0, 0, 0, 0, g,  0, 0, 0, 0],
 [0, 0, 0, 0, 0, -g, 0, 0, 0],
 [0, 0, 0, 0, 0,  0, 0, 0, 0]]),
 Matrix([
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 1]]))

In [29]:
C = h.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
D = h.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))

In [30]:
s_obs_index = [0, 1, 2, 4, 5, 6, 7, 8]
A_obs = A[s_obs_index, :][:, s_obs_index]
B_obs = B[s_obs_index, :]
C_obs = C[:, s_obs_index]
D_obs = D

In [31]:
A_obs

Matrix([
[0, 0, 0, 0,  0, 1, 0, 0],
[0, 0, 0, 0,  0, 0, 1, 0],
[0, 0, 0, 0,  0, 0, 0, 1],
[0, 0, 0, 0,  0, 0, 0, 0],
[0, 0, 0, 0,  0, 0, 0, 0],
[0, 0, 0, g,  0, 0, 0, 0],
[0, 0, 0, 0, -g, 0, 0, 0],
[0, 0, 0, 0,  0, 0, 0, 0]])

In [32]:
B_obs

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 1, 0, 0],
[1, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 1]])

In [33]:
C_obs

Matrix([
[(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2), (o_y_eq - y_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq

In [34]:
D_obs

Matrix([
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0],
[     0, -k_flow, 0, 0],
[k_flow,       0, 0, 0],
[     0,       0, 0, 0]])

In [35]:
x_obs = sym.Matrix([o_x, o_y, o_z - o_z_eq, theta, phi, v_x, v_y, v_z])
C_obs*x_obs + D_obs*u

Matrix([
[o_x*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + o_y*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[o_x*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + o_y*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[o_x*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + o_y*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2)],
[o_x*(o_x_eq - x_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2) + o_y*(o_y_eq - y_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2) + (o_z - o_z_eq)*(o_z

In [36]:
C_obs*x_obs + D_obs*u-y

Matrix([
[-d0 + o_x*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + o_y*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)],
[-d1 + o_x*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + o_y*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)],
[-d2 + o_x*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + o_y*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)*

In [41]:
anchors_x_sym = [x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7]
anchors_y_sym = [y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7]
anchors_z_sym = [z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7]
anchors_x = [-1.82, -3.57, 1.82, 2.67, -3.61, 1.84, 2.74, -1.85]
anchors_y = [2.18, 3.18, 2.21, -3.30, -3.30, -0.90, 3.18, -0.93]
anchors_z = [1.10, 2.24, 1.10, 2.20, 2.25, 1.10, 2.24, 1.09]

In [37]:
# C_obs = C_obs.subs(tuple(zip(anchors_x_sym, anchors_x))).subs(tuple(zip(anchors_y_sym, anchors_y))).subs(tuple(zip(anchors_z_sym, anchors_z)))
# C_obs = C_obs.subs(o_x_eq, 0).subs(o_y_eq, 0).subs(o_z_eq, 0.5).subs(k_flow, 0.01 * 30.0 / np.deg2rad(4.2))
C_obs

Matrix([
[(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), (o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), (o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), (o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2), 0, 0,             0,             0, 0],
[(o_x_eq - x_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq - z_3)**2), (o_y_eq - y_3)/sqrt((o_x_eq - x_3)**2 + (o_y_eq - y_3)**2 + (o_z_eq

In [42]:
A_arr = np.array(A_obs.subs(g, -9.81).tolist(), dtype=np.float64)
B_arr = np.array(B_obs.subs(g, -9.81).tolist(), dtype=np.float64)
C_arr = np.array(C_obs.subs(tuple(zip(anchors_x_sym, anchors_x))).subs(tuple(zip(anchors_y_sym, anchors_y))).subs(tuple(zip(anchors_z_sym, anchors_z))).subs(o_x_eq, 0).subs(o_y_eq, 0).subs(o_z_eq, 0.5).subs(k_flow, 0.01 * 30.0 / np.deg2rad(4.2)).tolist(), dtype=np.float64)

In [40]:
def obsv(A, C):
    W = C
    for i in range(1, A.shape[0]):
        W = np.vstack([W, C @ np.linalg.matrix_power(A, i)])
    return W
W_o = obsv(A_arr, C_arr)
print(f'      rank of W_o is: {np.linalg.matrix_rank(W_o)}')
print(f'"full rank" would be: {A_arr.shape[0]}')

np.set_printoptions(precision=1)
print(W_o)

      rank of W_o is: 8
"full rank" would be: 8
[[  0.6  -0.8  -0.2   0.    0.    0.    0.    0. ]
 [  0.7  -0.6  -0.3   0.    0.    0.    0.    0. ]
 [ -0.6  -0.8  -0.2   0.    0.    0.    0.    0. ]
 [ -0.6   0.7  -0.4   0.    0.    0.    0.    0. ]
 [  0.7   0.6  -0.3   0.    0.    0.    0.    0. ]
 [ -0.9   0.4  -0.3   0.    0.    0.    0.    0. ]
 [ -0.6  -0.7  -0.4   0.    0.    0.    0.    0. ]
 [  0.9   0.4  -0.3   0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0.    8.2   0.    0. ]
 [  0.    0.    0.    0.    0.    0.    8.2   0. ]
 [  0.    0.    1.    0.    0.    0.    0.    0. ]
 [  0.    0.    0.    0.    0.    0.6  -0.8  -0.2]
 [  0.    0.    0.    0.    0.    0.7  -0.6  -0.3]
 [  0.    0.    0.    0.    0.   -0.6  -0.8  -0.2]
 [  0.    0.    0.    0.    0.   -0.6   0.7  -0.4]
 [  0.    0.    0.    0.    0.    0.7   0.6  -0.3]
 [  0.    0.    0.    0.    0.   -0.9   0.4  -0.3]
 [  0.    0.    0.    0.    0.   -0.6  -0.7  -0.4]
 [  0.    0.    0.    0.    0.    

In [52]:
L_0, L_1, L_2, L_3, L_4, L_5, L_6, L_7, L_8, L_9, L_10 = sym.symbols('L_0, L_1, L_2, L_3, L_4, L_5, L_6, L_7, L_8, L_9, L_10')
L_11, L_12, L_13, L_14, L_15, L_16, L_17, L_18, L_19, L_20, L_21 = sym.symbols('L_11, L_12, L_13, L_14, L_15, L_16, L_17, L_18, L_19, L_20, L_21')
L_22, L_23, L_24, L_25, L_26, L_27, L_28, L_29, L_30, L_31, L_32 = sym.symbols('L_22, L_23, L_24, L_25, L_26, L_27, L_28, L_29, L_30, L_31, L_32')
L_33, L_34, L_35, L_36, L_37, L_38, L_39, L_40, L_41, L_42, L_43 = sym.symbols('L_33, L_34, L_35, L_36, L_37, L_38, L_39, L_40, L_41, L_42, L_43')
L_44, L_45, L_46, L_47, L_48, L_49, L_50, L_51, L_52, L_53, L_54 = sym.symbols('L_44, L_45, L_46, L_47, L_48, L_49, L_50, L_51, L_52, L_53, L_54')
L_55, L_56, L_57, L_58, L_59, L_60, L_61, L_62, L_63, L_64, L_65 = sym.symbols('L_55, L_56, L_57, L_58, L_59, L_60, L_61, L_62, L_63, L_64, L_65')
L_66, L_67, L_68, L_69, L_70, L_71, L_72, L_73, L_74, L_75, L_76 = sym.symbols('L_66, L_67, L_68, L_69, L_70, L_71, L_72, L_73, L_74, L_75, L_76')
L_77, L_78, L_79, L_80, L_81, L_82, L_83, L_84, L_85, L_86, L_87 = sym.symbols('L_77, L_78, L_79, L_80, L_81, L_82, L_83, L_84, L_85, L_86, L_87')
L_var_sym = sym.Matrix([[L_0, L_1, L_2, L_3, L_4, L_5, L_6, L_7, L_8, L_9, L_10],
                        [L_11, L_12, L_13, L_14, L_15, L_16, L_17, L_18, L_19, L_20, L_21],
                        [L_22, L_23, L_24, L_25, L_26, L_27, L_28, L_29, L_30, L_31, L_32],
                        [L_33, L_34, L_35, L_36, L_37, L_38, L_39, L_40, L_41, L_42, L_43],
                        [L_44, L_45, L_46, L_47, L_48, L_49, L_50, L_51, L_52, L_53, L_54],
                        [L_55, L_56, L_57, L_58, L_59, L_60, L_61, L_62, L_63, L_64, L_65],
                        [L_66, L_67, L_68, L_69, L_70, L_71, L_72, L_73, L_74, L_75, L_76],
                        [L_77, L_78, L_79, L_80, L_81, L_82, L_83, L_84, L_85, L_86, L_87]])

In [55]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K
Q = np.diag([
    1/0.049**2,
    1/0.072**2,
    1/0.046**2,
    1/0.114**2,
    1/0.063**2,
    1/0.048**2,
    1/0.07**2,
    1/0.038**2,
    1/1.696**2,             # n_x
    1/1.738**2,             # n_y
    1/0.148**2,             # r
])

R = np.diag([
    1/0.094**2,
    1/0.111**2,
    1/0.009**2, # o_z
    1/0.036**2, # theta
    1/0.030**2, # phi
    1/0.176**2, # v_x
    1/0.178**2, # v_y
    1/0.354**2, # v_z
])
L_2 = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T
print(L_2.tolist())


[[1.12707648705855, 0.5476728004584999, -1.0615881241413503, -0.22705939253042295, 0.5521059845228115, -1.6658726982264331, -0.4755268549752721, 2.071143650670068, 0.021468870601493426, -0.0016840796699426899, 0.03396661692088785], [-1.738162426062706, -0.6740358712502084, -1.6909174624896368, 0.3254181936459032, 0.7878964648099376, 1.195676799153298, -0.6579244075067123, 1.3521973430235452, -0.0019777992550057506, 0.022790100502690716, -0.018269759746850644], [-0.8179381353446243, -0.7226154439982491, -1.3565303235806452, -0.43358432416940296, -1.055131098765307, -1.9635185747847415, -1.0668322493240292, -2.1841243716682874, 0.0024293584737009205, -0.001088214634718461, 0.6031404742871412], [-0.19647989907609797, -0.09856264001587232, 0.1946635210026783, 0.03617026307544228, -0.11115485414170244, 0.28064544271884173, 0.08378407944223545, -0.3956183463009701, -0.014008854739422613, 0.00028130782638563777, -0.002395797816329901], [-0.22234364742970822, -0.08642764272198877, -0.228875313

In [48]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K
Q = np.diag([
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,             # n_x
    1,             # n_y
    1,             # r
])

R = np.diag([
    1,
    1,
    1, # o_z
    1, # theta
    1, # phi
    1, # v_x
    1, # v_y
    1, # v_z
])
L_1 = lqr(np.array(A_arr.tolist()).T, np.array(C_arr.tolist()).T, linalg.inv(R), linalg.inv(Q)).T
L_1_round = L_1.copy()
for i in range(L_1.shape[0]):
    for j in range(L_1.shape[1]):
        L_1_round[i,j] = round(L_1[i,j], 6)
print(L_1.tolist())

[[0.3180489943470645, 0.3553443337717232, -0.31585157789868507, -0.2982189814509199, 0.3509203825098959, -0.43880375885054024, -0.3067959493668199, 0.4346906747212534, 0.11790331190534352, -9.654711870720172e-06, 0.0035907444592605956], [-0.4140433503589006, -0.3423397612797527, -0.41552483450602606, 0.405793758565189, 0.35635821522408806, 0.23841012171915674, -0.3821835054177547, 0.2426216586331783, -9.665354708967095e-06, 0.11849182375122305, -0.01395588078308466], [-0.23465260273614175, -0.3980459263629877, -0.23716485961905617, -0.4570934079435634, -0.4095359783329232, -0.34540030810949485, -0.4506647630854788, -0.3308925346291107, 3.667091612311155e-05, -0.00012857175039766942, 1.1967500474500081], [-0.0013389045963223677, -0.0014997924714295811, 0.0013278869703700386, 0.0012415335824376356, -0.0014879763763421458, 0.0018368121665299632, 0.0012851484950959667, -0.0018377365930699306, -0.9999910181328046, 3.604321660515015e-08, 9.28577500702319e-06], [-0.0015156421660876616, -0.001

In [56]:
L_str = np.array2string(L_2,
                        formatter={'float_kind': lambda x: f'{x:12.4f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'L = {L_str}')

L = [[      1.1271       0.5477      -1.0616      -0.2271       0.5521      -1.6659      -0.4755       2.0711       0.0215      -0.0017       0.0340]
     [     -1.7382      -0.6740      -1.6909       0.3254       0.7879       1.1957      -0.6579       1.3522      -0.0020       0.0228      -0.0183]
     [     -0.8179      -0.7226      -1.3565      -0.4336      -1.0551      -1.9635      -1.0668      -2.1841       0.0024      -0.0011       0.6031]
     [     -0.1965      -0.0986       0.1947       0.0362      -0.1112       0.2806       0.0838      -0.3956      -0.0140       0.0003      -0.0024]
     [     -0.2223      -0.0864      -0.2289       0.0404       0.1036       0.1433      -0.0904       0.1831      -0.0003       0.0120      -0.0010]
     [      2.1142       1.0487      -2.0530      -0.4020       1.1373      -3.0543      -0.8959       4.1195       0.1251      -0.0033       0.0390]
     [     -2.7586      -1.0717      -2.7813       0.5065       1.2714       1.8208      -1.0933    

In [57]:
A_sym = sym.Matrix(A)
B_sym = sym.Matrix(B)
A_obs_sym = sym.Matrix(A_obs)
B_obs_sym = sym.Matrix(B_obs)
C_obs_sym = sym.Matrix(C_obs)
D_obs_sym = sym.Matrix(D_obs)
# L_0, L_1, L_2, L_3, L_4, L_5 = sym.symbols('L_0, L_1, L_2, L_3, L_4, L_5')
# L_sym = sym.Matrix([[0, 0, L_0],
#                     [L_1, 0, 0],
#                     [0, L_2, 0],
#                     [L_3, 0, 0],
#                     [0, L_4, 0],
#                     [0, 0, L_5]])
L_sym = sym.Matrix(L_2)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')
psi, theta, phi = sym.symbols('psi, theta, phi')
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
a_z, g, n_x, n_y, r = sym.symbols('a_z, g, n_x, n_y, r')
x_sym = sym.Matrix([o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z])
x_obs_sym = sym.Matrix([o_x, o_y, o_z, theta, phi, v_x, v_y, v_z])
u_sym = sym.Matrix([w_x, w_y, w_z, a_z - g])
y_sym = sym.Matrix([a_0, a_1, a_2, a_3, a_4, a_5, a_6, a_7, n_x, n_y, r])
x_dot_sym = A_sym * x_sym + B_sym * u_sym
x_obs_dot_sym = A_obs_sym * x_obs_sym + B_obs_sym * u_sym
y_err_sym = C_obs_sym * x_obs_sym + D_obs_sym * u_sym - y_sym
x_hat_dot_sym = x_obs_dot_sym - L_sym * y_err_sym

In [53]:
u_sym = sym.Matrix([w_x, w_y, w_z, a_z - g])
x_obs_dot = A_obs * x_obs + B_obs * u_sym
y_err_sym = C_obs * x_obs + D_obs * u_sym - y
# L_sym = sym.Matrix(L_1_round)
x_hat_dot_sym = x_obs_dot - L_var_sym * y_err_sym

In [54]:
x_hat_dot_sym

Matrix([
[              -L_0*(-d0 + o_x*(o_x_eq - x_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + o_y*(o_y_eq - y_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + (o_z - o_z_eq)*(o_z_eq - z_0)/sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2) + sqrt((o_x_eq - x_0)**2 + (o_y_eq - y_0)**2 + (o_z_eq - z_0)**2)) - L_1*(-d1 + o_x*(o_x_eq - x_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + o_y*(o_y_eq - y_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + (o_z - o_z_eq)*(o_z_eq - z_1)/sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2) + sqrt((o_x_eq - x_1)**2 + (o_y_eq - y_1)**2 + (o_z_eq - z_1)**2)) - L_10*(o_z - r) - L_2*(-d2 + o_x*(o_x_eq - x_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + o_y*(o_y_eq - y_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)**2) + (o_z - o_z_eq)*(o_z_eq - z_2)/sqrt((o_x_eq - x_2)**2 + (o_y_eq - y_2)**2 + (o_z_eq - z_2)

default c and o hover
default c and o hover with loco
custom c and default o hover with loco
implement observer offline

In [58]:
Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10 = sym.symbols('Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10')
Y_err = sym.Matrix([Y_0, Y_1, Y_2, Y_3, Y_4, Y_5, Y_6, Y_7, Y_8, Y_9, Y_10])
output_list = (x_obs_dot_sym - L_sym * Y_err).tolist()
output_list

[[-1.12707648705855*Y_0 - 0.5476728004585*Y_1 - 0.0339666169208879*Y_10 + 1.06158812414135*Y_2 + 0.227059392530423*Y_3 - 0.552105984522811*Y_4 + 1.66587269822643*Y_5 + 0.475526854975272*Y_6 - 2.07114365067007*Y_7 - 0.0214688706014934*Y_8 + 0.00168407966994269*Y_9 + v_x],
 [1.73816242606271*Y_0 + 0.674035871250208*Y_1 + 0.0182697597468506*Y_10 + 1.69091746248964*Y_2 - 0.325418193645903*Y_3 - 0.787896464809938*Y_4 - 1.1956767991533*Y_5 + 0.657924407506712*Y_6 - 1.35219734302355*Y_7 + 0.00197779925500575*Y_8 - 0.0227901005026907*Y_9 + v_y],
 [0.817938135344624*Y_0 + 0.722615443998249*Y_1 - 0.603140474287141*Y_10 + 1.35653032358065*Y_2 + 0.433584324169403*Y_3 + 1.05513109876531*Y_4 + 1.96351857478474*Y_5 + 1.06683224932403*Y_6 + 2.18412437166829*Y_7 - 0.00242935847370092*Y_8 + 0.00108821463471846*Y_9 + v_z],
 [0.196479899076098*Y_0 + 0.0985626400158723*Y_1 + 0.0023957978163299*Y_10 - 0.194663521002678*Y_2 - 0.0361702630754423*Y_3 + 0.111154854141702*Y_4 - 0.280645442718842*Y_5 - 0.08378407

In [59]:
for i in range(len(output_list)):
    line = str(output_list[i])
    output_list[i] = line.replace("*", "f*" )
    print(output_list[i])
    print("\n")



[-1.12707648705855f*Y_0 - 0.5476728004585f*Y_1 - 0.0339666169208879f*Y_10 + 1.06158812414135f*Y_2 + 0.227059392530423f*Y_3 - 0.552105984522811f*Y_4 + 1.66587269822643f*Y_5 + 0.475526854975272f*Y_6 - 2.07114365067007f*Y_7 - 0.0214688706014934f*Y_8 + 0.00168407966994269f*Y_9 + v_x]


[1.73816242606271f*Y_0 + 0.674035871250208f*Y_1 + 0.0182697597468506f*Y_10 + 1.69091746248964f*Y_2 - 0.325418193645903f*Y_3 - 0.787896464809938f*Y_4 - 1.1956767991533f*Y_5 + 0.657924407506712f*Y_6 - 1.35219734302355f*Y_7 + 0.00197779925500575f*Y_8 - 0.0227901005026907f*Y_9 + v_y]


[0.817938135344624f*Y_0 + 0.722615443998249f*Y_1 - 0.603140474287141f*Y_10 + 1.35653032358065f*Y_2 + 0.433584324169403f*Y_3 + 1.05513109876531f*Y_4 + 1.96351857478474f*Y_5 + 1.06683224932403f*Y_6 + 2.18412437166829f*Y_7 - 0.00242935847370092f*Y_8 + 0.00108821463471846f*Y_9 + v_z]


[0.196479899076098f*Y_0 + 0.0985626400158723f*Y_1 + 0.0023957978163299f*Y_10 - 0.194663521002678f*Y_2 - 0.0361702630754423f*Y_3 + 0.111154854141702f*Y_

In [47]:
x_dot_sym

Matrix([
[    v_x],
[    v_y],
[    v_z],
[    w_z],
[    w_y],
[    w_x],
[g*theta],
[ -g*phi],
[a_z - g]])

In [48]:
x_hat_dot_sym


Matrix([
[                             0.318048994347065*a_0 + 0.355344333771723*a_1 - 0.315851577898685*a_2 - 0.29821898145092*a_3 + 0.350920382509896*a_4 - 0.43880375885054*a_5 - 0.30679594936682*a_6 + 0.434690674721253*a_7 + 9.65471187072017e-6*k_flow*w_x + 0.117903311905344*k_flow*w_y + 0.117903311905344*n_x - 9.65471187072017e-6*n_y - 2.0000819388963*o_x - 0.00280219058557235*o_y + 0.00453090403648562*o_z + 0.0035907444592606*r + 0.0349482624584663*v_x + 7.90248918010174e-5*v_y],
[                           -0.414043350358901*a_0 - 0.342339761279753*a_1 - 0.415524834506026*a_2 + 0.405793758565189*a_3 + 0.356358215224088*a_4 + 0.238410121719157*a_5 - 0.382183505417755*a_6 + 0.242621658633178*a_7 - 0.118491823751223*k_flow*w_x - 9.6653547089671e-6*k_flow*w_y - 9.6653547089671e-6*n_x + 0.118491823751223*n_y - 0.00280453047536086*o_x - 1.83091493368707*o_y - 0.0158698944774888*o_z - 0.0139558807830847*r + 7.91120046172444e-5*v_x + 0.0301312274638448*v_y],
[               -0.2346526027